<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/karen_rosero/Aula_8_Exerc%C3%ADcio_KarenRosero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Karen Rosero'
print(f'Meu nome é {nome}')

Meu nome é Karen Rosero


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [3]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

In [6]:
!pip install torch

## Importação dos pacotes

In [2]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [3]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 19:04:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:02:00.0  On |                  N/A |
| 48%   62C    P2    48W / 250W |    611MiB / 12063MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [5]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        # Escreva seu código aqui
        self.tokenizer = tokenizer
        self.context_size = context_size
        self.features, self.y = self._token_window(texts)

    def _token_window(self, texts):
        feat = []
        y = []
        # bucle para ler cada uma das frases de entrada
        for text in texts:
          # tokeniza uma frase
          tokens_from_text = tokenize(text, self.tokenizer)
          # validar que o número de tokens na frase seja maior do que context_size
          if len(tokens_from_text) > self.context_size:
              # bucle para selecionar só o número de tokens indicado por context size
              for i in range(len(tokens_from_text)-self.context_size): 
                feat.append(tokens_from_text[i:i+self.context_size])
                y.append(tokens_from_text[i+self.context_size])
        return torch.tensor(feat).long(), torch.tensor(y).long()

    def __len__(self):
        # Escreva seu código aqui
        return len(self.features)

    def __getitem__(self, idx):
        # Escreva seu código aqui
        return self.features[idx], self.y[idx]

## Testando se a implementação do MyDataset está correta

In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [7]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

File ‘sample_brwac.txt’ already there; not retrieving.



In [8]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

#print('Truncating for debugging purposes.')
#texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]
print(len(training_texts), len(valid_texts))

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

24800 100


In [9]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [23]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        super(LanguageModel, self).__init__()

        self.context_size = context_size
        self.embedding_dim = embedding_dim

        # Words embedding C(W) --> params (V,D)
        self.C_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # Positions embedding P() --> params (L,D)
        self.P_embeddings = nn.Embedding(self.context_size, embedding_dim)
        # Linear projections, input D output D
        self.W_Q = nn.Linear(embedding_dim, embedding_dim)
        self.W_K = nn.Linear(embedding_dim, embedding_dim)
        self.W_V = nn.Linear(embedding_dim, embedding_dim)
        self.W_0 = nn.Linear(embedding_dim, embedding_dim)
        # Fully connected layers
        self.dense_layers = nn.Sequential(
            nn.Linear(embedding_dim, 2*embedding_dim),
            nn.ReLU(),
            nn.Linear(2*embedding_dim, embedding_dim))
        
        #adotei essa linha do notebook do Patrick Ferreira
        self.linear_output = nn.Linear(embedding_dim, vocab_size)

        self.soft = nn.Softmax(dim=-1)

        self.norm = nn.LayerNorm(embedding_dim)

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size) = B,L
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        batch_size = inputs.shape[0] 
        # posições para cada vetor de contexto = (B,L)
        positions_mat = torch.arange(self.context_size).view(1, -1).repeat(inputs.shape[0], 1).to(device)
        # C(Wn) --> (B,L,D)
        C_emb = self.C_embeddings(inputs)
        # P(n) --> (B,L,D)
        P_emb = self.P_embeddings(positions_mat)
        # C(Wn) + P(n) = X(Wn)
        x = C_emb + P_emb
        # Q(W) = X(W)W_Q
        Q = self.W_Q(x)
        # K(W) = X(W)W_K
        K = self.W_K(x)
        # V(W) = X(W)W_V
        V = self.W_V(x)
        # calculo de scores
        scores = torch.matmul(Q, K.transpose(1, 2))
        probs = self.soft(scores)
        E = self.W_0(self.norm(self.norm(x + torch.matmul(probs,V))))
        logits = self.dense_layers(E[:,-1,:].flatten(start_dim=1))
        logi_n = self.linear_output(self.norm(logits + E[:, -1, :].flatten(start_dim=1)))
        return logi_n

## Teste o modelo com um exemplo

In [24]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [25]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 7790434


## Assert da Perplexidade


In [26]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              35453
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [28]:
max_examples = 100_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 35642.39, valid ppl: 33769.43
10000 steps; 10240000 examples so far; train ppl: 579.39, valid ppl: 359.97
20000 steps; 20480000 examples so far; train ppl: 320.95, valid ppl: 291.25
30000 steps; 30720000 examples so far; train ppl: 277.12, valid ppl: 266.84
40000 steps; 40960000 examples so far; train ppl: 248.07, valid ppl: 233.59
50000 steps; 51200000 examples so far; train ppl: 225.13, valid ppl: 216.95
60000 steps; 61440000 examples so far; train ppl: 209.05, valid ppl: 206.81
70000 steps; 71680000 examples so far; train ppl: 201.62, valid ppl: 199.72
80000 steps; 81920000 examples so far; train ppl: 196.09, valid ppl: 195.03
90000 steps; 92160000 examples so far; train ppl: 187.54, valid ppl: 190.45


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [30]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 180.32862339058548


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [38]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz com
Eu gosto de comer pizza pois me faz com que
Eu gosto de comer pizza pois me faz com que o
Eu gosto de comer pizza pois me faz com que o governo
Eu gosto de comer pizza pois me faz com que o governo federal
Eu gosto de comer pizza pois me faz com que o governo federal,
Eu gosto de comer pizza pois me faz com que o governo federal, que
Eu gosto de comer pizza pois me faz com que o governo federal, que é
Eu gosto de comer pizza pois me faz com que o governo federal, que é o
Eu gosto de comer pizza pois me faz com que o governo federal, que é o que
Eu gosto de comer pizza pois me faz com que o governo federal, que é o que se
Eu gosto de comer pizza pois me faz com que o governo federal, que é o que se refere
Eu gosto de comer pizza pois me faz com que o governo federal, que é o que se refere a
Eu gosto de comer pizza pois me faz com que o governo federal, que é o que se refere a sua
Eu gosto de comer pizza pois me faz com que o governo federal, que é